In [1]:
import os
import io
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import anthropic
from IPython.display import Markdown, display, update_display
import subprocess
import sys
import numba
import numpy as np

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [3]:
GPT_MODEL = "gpt-4o-mini"
Claude_Model = "claude-opus-4-1-20250805"
openai = OpenAI()
claude = anthropic.Anthropic()

In [4]:
system_message = "You are an assistant that add comments to the each line of a given Python or R code"
system_message += "Respond with the orginal python code without doing any changes to its coding part."
system_message += "The output shouldn't include '```python' and '```' parts"
system_message += "There should be a linespace between previous code line and the comment"
system_message += "Finally, the code should be in light blue colour, where comments are in italic white colour"

def user_prompt_for(code):
    user_prompt =  "Add comments to each line of this python code.\n\n"
    user_prompt += code
    return user_prompt

In [5]:
def gpt_model(code):

    response = openai.chat.completions.create(
        model = GPT_MODEL,
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt_for(code)}
        ],
        stream = True
    )
    reply = ""
    for chunk in response:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [6]:
def claude_model(code):

    result = claude.messages.stream(
        model = Claude_Model,
        max_tokens = 2000,
        system = system_message,
        messages = [{"role": "user", "content": user_prompt_for(code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply

In [7]:
def optimize(code, model):
    if model == "GPT":
        result = gpt_model(code)
    elif model == "Claude":
        result = claude_model(code)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far 

In [8]:
with gr.Blocks() as ui:
    with gr.Row():
        code = gr.Textbox(label="Insert your python or R code:", lines=10)
        code_with_comments = gr.Textbox(label="Code with comments:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Add Comments to the code")

    convert.click(optimize, inputs=[code, model], outputs=[code_with_comments])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
